### Import Required Libraries and Set Up Environment Variables

In [6]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [7]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [8]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f"{url}api-key={nyt_api_key }&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [9]:
# Get the response
response = requests.get(query_url)
response

<Response [200]>

In [10]:
# Retrieve reviews
reviews = response.json()

# Print results in JSON format
print(json.dumps(reviews, indent=4))

{
    "status": "OK",
    "copyright": "Copyright (c) 2024 The New York Times Company. All Rights Reserved.",
    "response": {
        "docs": [
            {
                "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
                "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
                "source": "The New York Times",
                "headline": {
                    "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
                    "kicker": null,
                    "content_kicker": null,
                    "print_headline": "The Attachment Diaries",
                    "name": null,
                    "seo": null,
                    "sub": null
                },
                "keywords": [
                    {
                        "name": "subject",
                        "value": "Movies",
                        "ra

In [11]:
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from urllib.parse import quote
from pandas import json_normalize

load_dotenv()
nyt_api_key = os.getenv("NYT_API_KEY")

# Function to extract keywords (assuming the structure from NYT response)
def extract_keywords(keywords_list):
    return ", ".join([keyword['value'] for keyword in keywords_list])

# Base parameters for NYT API
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"
filter_query = quote('section_name:"Movies" AND type_of_material:"Review" AND headline:"love"')
sort = "newest"
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"
begin_date = "20130101"
end_date = "20230531"


reviews_list = []

# Loop through pages of results, with rate limiting
for page in range(20):
    query_url = f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}&fq={filter_query}&sort={sort}&fl={field_list}&page={page}"
    
    try:
        response = requests.get(query_url)
        reviews = response.json()

        for review in reviews["response"]["docs"]:
            reviews_list.append(review)
        
        # Print a progress message after each page
        print(f"Retrieved page {page + 1}")  

        # Wait 12 seconds (be mindful of the rate limits!)
        time.sleep(12)
    except KeyError:  # This handles the case where a page has no results
        print(f"No results found on page {page + 1}. Exiting loop.")
        break
    except Exception as e:  # Catching general errors for debugging
        print(f"An error occurred on page {page + 1}: {e}")
        break

# Print the first 5 results (for debugging/inspection)
print(json.dumps(reviews_list[:5], indent=4))

# Convert to DataFrame and extract movie titles and keywords
reviews_df = json_normalize(reviews_list)
reviews_df["title"] = reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)

# Get unique movie titles for TMDB query
titles = reviews_df["title"].tolist()

print(titles)


Retrieved page 1
Retrieved page 2
Retrieved page 3
Retrieved page 4
Retrieved page 5
Retrieved page 6
Retrieved page 7
Retrieved page 8
Retrieved page 9
Retrieved page 10
Retrieved page 11
Retrieved page 12
Retrieved page 13
Retrieved page 14
Retrieved page 15
Retrieved page 16
Retrieved page 17
Retrieved page 18
Retrieved page 19
Retrieved page 20
[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "s

In [4]:
# Create an empty list to store the reviews


# loop through pages 0-19

    # create query with a page number
    # API results show 10 articles at a time

    
    # Make a "GET" request and retrieve the JSON

    
    # Add a twelve second interval between queries to stay within API query limits

    
    # Try and save the reviews to the reviews_list

        # loop through the reviews["response"]["docs"] and append each review to the list

        # Print the page that was just retrieved


        # Print the page number that had no results then break from the loop


Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [13]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4)) 


[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [15]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = json_normalize(reviews_list)
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [17]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df["title"] = reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])

reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children


In [18]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)


In [20]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df['title'].tolist()

titles

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [10]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [21]:
load_dotenv()
tmdb_api_key = os.getenv("TMDB_API_KEY")

# Create an empty list to store the movie data
tmdb_movies_list = []
request_counter = 1

# Loop through titles, making requests with rate limiting
for title in titles:
    # Build the search query URL
    search_url = f"https://api.themoviedb.org/3/search/movie?query={quote(title)}&api_key={tmdb_api_key}"

    try:
        # Search for movie ID
        search_response = requests.get(search_url)
        search_data = search_response.json()
        movie_id = search_data['results'][0]['id']  # Get the first matching result's ID
        
        # Get detailed movie information
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_response = requests.get(movie_url)
        movie_data = movie_response.json()

        # Extract specific details
        genres = [genre['name'] for genre in movie_data['genres']]
        spoken_languages = [lang['english_name'] for lang in movie_data['spoken_languages']]
        production_countries = [country['name'] for country in movie_data['production_countries']]

        # Create dictionary of movie data
        movie_dict = {
            "title": movie_data['title'],
            "original_title": movie_data['original_title'],
            "budget": movie_data['budget'],
            "original_language": movie_data['original_language'],
            "homepage": movie_data['homepage'],
            "overview": movie_data['overview'],
            "popularity": movie_data['popularity'],
            "runtime": movie_data['runtime'],
            "revenue": movie_data['revenue'],
            "release_date": movie_data['release_date'],
            "vote_average": movie_data['vote_average'],
            "vote_count": movie_data['vote_count'],
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries
        }
        
        # Add movie data to the list
        tmdb_movies_list.append(movie_dict)

        # Success message
        print(f"Found movie: {movie_data['title']}")

    except (KeyError, IndexError): # Handle cases where movie not found or details are missing
        print(f"Movie '{title}' not found or data is incomplete in TMDB.")
    except Exception as e:  # Catching general errors for debugging
        print(f"An error occurred while fetching '{title}': {e}") 
 
    # Rate limiting: Pause after every 50 requests
    if request_counter % 50 == 0:
        print("Sleeping for 60 seconds to respect rate limits...")
        time.sleep(60)
    request_counter += 1  

# Preview the first 5 results (for debugging/inspection)
print(json.dumps(tmdb_movies_list[:5], indent=4))

# Convert to DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)


Found movie: The Attachment Diaries
Movie 'What’s Love Got to Do With It?’ Probably a Lo' not found or data is incomplete in TMDB.
Found movie: You Can Live Forever
Found movie: A Tourist's Guide to Love
Found movie: Other People's Children
Found movie: One True Loves
Found movie: The Lost Weekend: A Love Story
Found movie: A Thousand and One
Found movie: Your Place or Mine
Found movie: Love in the Time of Fentanyl
Found movie: Pamela, A Love Story
Found movie: In from the Side
Found movie: After Love
Found movie: Alcarràs
Found movie: Nelly & Nadine
Found movie: Lady Chatterley's Lover
Found movie: The Sound of Christmas
Found movie: The Inspection
Found movie: Bones and All
Found movie: My Policeman
Found movie: About Fate
Found movie: Waiting for Bojangles
Found movie: I Love My Dad
Found movie: A Love Song
Found movie: Alone Together
Found movie: Art of Love
Found movie: 2099: The Soldier Protocol
Found movie: Thor: Love and Thunder
Found movie: Both Sides of the Blade
Found movie:

In [12]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4)) 

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "genre": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "language": "es",
        "spoken_languages": [
            "Spanish"
        ],
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 0.708,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 4.0,
        "vote_count": 2,
        "production_countries": [
            "Argentina"
        ]
    },
    

In [37]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

tmdb_df.head()


,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",3.871,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",35.245,96,15055,2023-03-24,6.474,38,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",30.992,96,0,2023-04-21,6.287,162,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",12.635,104,84178,2022-09-21,6.818,190,"[Drama, Comedy]","[French, English]",[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,12.922,100,37820,2023-04-07,6.500,75,"[Romance, Comedy, Drama]","[English, Spanish]","[Czech Republic, United States of America]"


### Merge and Clean the Data for Export

In [38]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_df,reviews_df, on="title", how="left", suffixes=("_nyt", "_tmdb"))

merged_df.columns

Index(['title', 'original_title', 'budget', 'original_language', 'homepage',
       'overview', 'popularity', 'runtime', 'revenue', 'release_date',
       'vote_average', 'vote_count', 'genres', 'spoken_languages',
       'production_countries', 'web_url', 'snippet', 'source', 'keywords',
       'pub_date', 'word_count', 'headline.main', 'headline.kicker',
       'headline.content_kicker', 'headline.print_headline', 'headline.name',
       'headline.seo', 'headline.sub', 'byline.original', 'byline.person',
       'byline.organization'],
      dtype='object')

In [48]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'spoken_languages', 'production_countries']

# Create a list of characters to remove
characters_to_remove = ['[', ']', "'"]

# Loop through the list of columns to fix
for col in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[col] = merged_df[col].astype(str)
    
    # Loop through characters to remove
    for char in characters_to_remove:
        merged_df[col] = merged_df[col].str.replace(char, '', regex=False)

# Display the fixed DataFrame
print(merged_df.head().to_markdown(index=False, numalign="left", stralign="left"))



| title                     | original_title            | budget   | original_language   | homepage                                                | overview                                                                                                                                                                                                                                                                                                                                                                                             | popularity   | runtime   | revenue   | release_date   | vote_average   | vote_count   | genres                           | spoken_languages    | production_countries                     | web_url                                                                      | snippet                                                                                                               | source             | keywords                                            

In [49]:
merged_df.columns

Index(['title', 'original_title', 'budget', 'original_language', 'homepage',
       'overview', 'popularity', 'runtime', 'revenue', 'release_date',
       'vote_average', 'vote_count', 'genres', 'spoken_languages',
       'production_countries', 'web_url', 'snippet', 'source', 'keywords',
       'pub_date', 'word_count', 'headline.main', 'headline.kicker',
       'headline.content_kicker', 'headline.print_headline', 'headline.name',
       'headline.seo', 'headline.sub', 'byline.original', 'byline.person',
       'byline.organization'],
      dtype='object')

In [50]:
# Drop "byline.person" column
merged_df.drop(columns=["byline.person"], inplace=True, errors='ignore')

Index(['title', 'original_title', 'budget', 'original_language', 'homepage',
       'overview', 'popularity', 'runtime', 'revenue', 'release_date',
       'vote_average', 'vote_count', 'genres', 'spoken_languages',
       'production_countries', 'web_url', 'snippet', 'source', 'keywords',
       'pub_date', 'word_count', 'headline.main', 'headline.kicker',
       'headline.content_kicker', 'headline.print_headline', 'headline.name',
       'headline.seo', 'headline.sub', 'byline.original',
       'byline.organization'],
      dtype='object')

In [51]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)


In [52]:
# Export data to CSV without the index
merged_df.to_csv("nyt_movie_reviews_tmdb.csv", index=False)
